## Build a simple LLM application with chat models and prompt templates

In [1]:
pip install -q google-genai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from google import genai

# Access the environment variable
genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))

### Using Language Models

First up, let's learn how to use a language model by itself. **LangChain** supports many different language models.

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-lite-preview-02-05")

Let's first use the model directly. `ChatModels` are instances of LangChain `Runnables`, which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of messages to the `.invoke` method.

In [5]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage("Translate the user sentence to French!"),
    HumanMessage("Hi!, My name is Sarath Rachuri")
]

In [6]:
llm.invoke(messages)

AIMessage(content="Bonjour ! Je m'appelle Sarath Rachuri.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-35498bae-0ece-408c-bd28-ea887100b215-0', usage_metadata={'input_tokens': 17, 'output_tokens': 11, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})

LangChain also supports chat model inputs via strings or OpenAI format.

In [7]:
llm.invoke([{"role": "assistant", "content": "Translate the user sentence into Italian"}, {"role": "user", "content": "Hello"}])

AIMessage(content='Ciao', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-714e1aff-0b41-497b-9901-a545694e7581-0', usage_metadata={'input_tokens': 9, 'output_tokens': 1, 'total_tokens': 10, 'input_token_details': {'cache_read': 0}})

### Streaming

Since chat models are **Runnables**, they expose a standard interface that includes `async` and `streaming` modes of invocation. Let's stream individual tokens from a chat model.

In [8]:
for token in llm.stream(messages):
    print(token.content, end="|")

Bonjour| ! Je m'appelle Sarath Rachuri.|

### Prompt Templates

Currently, we pass a list of messages directly into the language model. But in real world applications, messages are built from user input and application logic, which transforms raw input into a format suitable for the model.

**Prompt templates** in LangChain simplify this process. They take raw user input and return a formatted prompt ready for the language model.

Let's create a prompt template that uses two variables:
- `language`: The language to translate text into
- `text`: The text to translate

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the user text from English into {language}"

# using tuples
prompt_template = ChatPromptTemplate.from_messages(
    [("assistant", system_template), ("user", "{text}")]
)

# using Dictionary
prompt = prompt_template.invoke({"language": "Spanish", "text": "Hi, How are you"})

# to check prompt object
prompt

ChatPromptValue(messages=[AIMessage(content='Translate the user text from English into Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi, How are you', additional_kwargs={}, response_metadata={})])

In [10]:
# To access the messages directly
prompt.to_messages()

[AIMessage(content='Translate the user text from English into Spanish', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, How are you', additional_kwargs={}, response_metadata={})]

In [11]:
# finally, invoke the chat model
response = llm.invoke(prompt)
print(response.content)

Hola, ¿cómo estás?
